In [7]:
# %pip install sentence_transformers
# %pip install fastapi
# %pip install uvicorn
# %pip install scikit-learn
# %pip install joblib

# import sentence_transformers
# from sentence_transformers import SentenceTransformer

In [4]:
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.linear_model import LogisticRegression
from sklearn.multioutput import MultiOutputClassifier
from sentence_transformers import SentenceTransformer
from sklearn.metrics import classification_report, roc_curve, auc
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np

# 1. Cargar el dataset
df = pd.read_csv('../data/processed/books.csv')

# 2. Reducir el tamaño del dataset para pruebas iniciales
df = df.sample(n=100, random_state=42)  # Trabajar con una muestra de 100 libros

# 3. Combinar título y blurb como entrada textual
df['text'] = df['book_title'].fillna('') + '. ' + df['blurb'].fillna('')

# 4. Procesar los tags: convertirlos en listas
df['tags'] = df['tags'].apply(lambda x: [t.strip() for t in str(x).split(',')])

# 5. Binarización multilabel
mlb = MultiLabelBinarizer()
Y = mlb.fit_transform(df['tags'])

# 6. Dividir los datos en entrenamiento y prueba
while True:
    X_train, X_test, Y_train, Y_test = train_test_split(df['text'], Y, test_size=0.2, random_state=42)
    if (Y_train.sum(axis=0) > 0).all():
        break

# 7. Generar embeddings con un modelo más rápido y en lotes
model = SentenceTransformer('distiluse-base-multilingual-cased-v1')  # Modelo más rápido

def generate_embeddings(texts, batch_size=16):
    embeddings = []
    for i in range(0, len(texts), batch_size):
        batch = texts[i:i + batch_size]
        embeddings.extend(model.encode(batch, show_progress_bar=False))
    return np.array(embeddings)

X_train_embeddings = generate_embeddings(X_train.tolist())
X_test_embeddings = generate_embeddings(X_test.tolist())

# 8. Entrenar modelo multilabel
clf = MultiOutputClassifier(LogisticRegression(max_iter=500))
clf.fit(X_train_embeddings, Y_train)

# 9. Evaluar el modelo
Y_pred = clf.predict(X_test_embeddings)
print(classification_report(Y_test, Y_pred, target_names=mlb.classes_))

# 10. Generar curvas ROC para las 5 etiquetas más frecuentes
top_labels = np.argsort(Y.sum(axis=0))[-5:]  # Seleccionar las 5 etiquetas más frecuentes
plt.figure(figsize=(10, 6))
for i in top_labels:
    fpr, tpr, _ = roc_curve(Y_test[:, i], clf.predict_proba(X_test_embeddings)[:, i][:, 1])
    roc_auc = auc(fpr, tpr)
    plt.plot(fpr, tpr, label=f'{mlb.classes_[i]} (AUC = {roc_auc:.2f})')

plt.plot([0, 1], [0, 1], 'k--', lw=2)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curves for Top 5 Labels')
plt.legend(loc='best')
plt.grid()
plt.show()

KeyboardInterrupt: 

In [9]:
# Ejemplo de predicción de un libro inventado
new_titles = ["La conspiración del universo"]
new_blurbs = ["Una historia que entrelaza ciencia, fe y filosofía para revelar los secretos de la creación."]
predicted_tags = predict_tags(new_titles, new_blurbs)
print(predicted_tags)

[('astronomy', 'philosophy', 'religion', 'science', 'to-read')]


In [10]:
# Ejemplo generación de embeddings para la "Isla del Tesoro" en inglés
new_titles = ["Treasure Island"]
new_blurbs = ["Treasure Island is an adventure novel by Scottish author Robert Louis Stevenson, narrating a tale of buccaneers and buried gold."]
predicted_tags = predict_tags(new_titles, new_blurbs)
print(predicted_tags)

[('catholic', 'classics', 'fiction', 'literary-criticism', 'literature', 'to-read')]


In [11]:
# Ejemplo generación de embeddings para la "Isla del Tesoro" en español
new_titles = ["La isla del tesoro"]
new_blurbs = ["La isla del tesoro es una novela de aventuras del autor escocés Robert Louis Stevenson, que narra una historia de bucaneros y oro enterrado."]
predicted_tags = predict_tags(new_titles, new_blurbs)
print(predicted_tags)

[('catholic', 'classics', 'fiction', 'literary-criticism', 'literature', 'to-read')]


In [12]:
# Ejemplo generación de embeddings para "El Quijote"
new_titles = ["Don Quijote de la Mancha"]
new_blurbs = ["Don Quijote de la Mancha es una novela escrita por el español Miguel de Cervantes Saavedra. Publicada su primera parte con el título de El ingenioso hidalgo don Quijote de la Mancha a comienzos de 1605, es una de las obras más destacadas de la literatura española y la literatura universal."]
predicted_tags = predict_tags(new_titles, new_blurbs)
print(predicted_tags)

[('catholic', 'classics', 'literature', 'spain', 'to-read')]
